# ACETONE tutorial #1

**Generating the code from a given network**

In this notebook, we generate the C code corresponding to a **Acas COC (?)** neural network, described in two formats: *ONNX* and *NNet*. We then use a random dataset (generated by the package) to infere our code and checking that the values remain consistent.

In the first part of the notebook, we instantiate the main class of ACETONE and use it to generate code.

In the second part, we compile the generated code and run it, before comparing the several outputs given by the package.

We will show that ACETONE remains consistent regardless of the format of the input.

* When running this notebook on Colab, we need to install ACETONE 
* If you run this notebook locally, run it in the environment in which you installed ACETONE

In [ ]:
#TODO check install on collab
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import sys  # noqa: avoid having this import removed by pycln

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install git+https://github.com/onera/acetone.git

In [ ]:
# Cleaning the working environment
from pathlib import Path
from os import remove, listdir
files_directories = [Path("demo_acas_onnx"), Path("demo_acas_nnet")]

for directory in files_directories:
    if directory.exists():
        for file in listdir(directory):
            remove(directory / file)

## Imports

Only one import is needed for a basic usage of ACETONE: the **CodeGenerator** class.

In [ ]:
from acetone_nnet import CodeGenerator

This class implements a local traduction of the neural networkof interest, storing the layers, architectures and parameters of the network. Moreover, the class implements a python inference, with a definite (given or randomly generated) set of inputs, offering a reference against which the C code can be compared.

## Generating code

The network we'll use as an exemple if an ACAS with 6 Dense layer, each separated by a Relu function.

In [ ]:
# TODO Image of the network

### Instantiating a **CodeGenerator** element

One parameter is truly vital for a **CodeGenerator** element: *model_path*, the path to the model of interest.

Other parameters can be given to personalize the generated code, such as the name of the generated function, or the number of the number of datasets on which we are going to perform inference.

Finally, the last parameter to set up is the path to the directory in which the code will be generated.

In [ ]:
model_path = "../tests/models/acas/acas_COC/nn_acas_COC.nnet"

function_name = "demo_acas"
nb_tests = 1

nnet_output_path = "demo_acas_nnet"

In [ ]:
# Create an ACETONE CodeGenerator from the model
generator = CodeGenerator(file=model_path,
                            function_name=function_name,
                            nb_tests=nb_tests)

### Generating the C code

We use the *generate_c_file* methode for generating the code. This methode create, in the directory *output_path*, several files containing elements of the code:

* *global_vars.c*  : Initialization of model parameters

* *inference.h*    : Header declaration of the model parameters and the inference function
* *inference.c*    : Definition of the inference function
* *test_dataset.h* : Declaration of global prameters (input size, number of test, ...) and of the test inputs
* *test_dataset.c* : Initialization of the test inputs
* *main.c*         : Main function, calls the inference on the input and write the result in a file
* *Makefile*       : Makefile to compile the C code

In [ ]:
generator.generate_c_files(nnet_output_path)

### Importing the ONNX model

We do the same operation as before, but this time with an ONNX model.

In [ ]:
model_path = "../tests/models/acas/acas_COC/nn_acas_COC.onnx"
onnx_output_path = "demo_acas_onnx"

#
onnx_generator = CodeGenerator(file=model_path,
                                function_name=function_name,
                                nb_tests=nb_tests)

In [ ]:
onnx_generator.generate_c_files(onnx_output_path)

### Generating the Python output

We use the *compute_inference* methode to compute a first evaluation off the inference function on the inputs, using ACETONE's python implementation of the layers. 


In [ ]:
# Computing the inference for the nnet model
nnet_output = generator.compute_inference(nnet_output_path)
print(nnet_output)

In [ ]:
# Computing the inference for the onnx model
onnx_output = onnx_generator.compute_inference(onnx_output_path)
print(onnx_output)

## Compiling and running the generated code

Alongside the neural network's code, a main file is generated as a way to run and test the code. The provided Makefile gives the flags to use for the compilation.

In [ ]:
! make -C demo_acas_nnet all

To run the executable file, add as parameter the path to the text file in which the ouptut will be written. 

In [ ]:
! ./demo_acas_nnet/demo_acas ./demo_acas_nnet/output_c.txt

Similary, we compile and run the code from the onnx model.

In [ ]:
! make -C demo_acas_onnx all

In [ ]:
! ./demo_acas_onnx/demo_acas ./demo_acas_onnx/output_c.txt

## Comparing two ouptuts

To verify if the two code did give the same value, we use the terminal command *acetone_comapre*. Thios command takes as input the path to two ouptut files (C or python) and the number of test done (here 1), and compare them term to term, returning both the max absolute error and the max relative error.

In [ ]:
! acetone_compare ./demo_acas_nnet/output_python.txt ./demo_acas_nnet/output_c.txt 1 --precision=float

In [ ]:
! acetone_compare ./demo_acas_onnx/output_c.txt ./demo_acas_nnet/output_c.txt 1 --precision=float

In [ ]:
! acetone_compare ./demo_acas_nnet/output_python.txt ./demo_acas_onnx/output_python.txt 1 --precision=float

# TODO
- Delete generated code before generation
- Rename the nnet to nnet
- Add calls to
  - compiler
  - run
  - compare nnet and onnx outputs in C
  - compare nnet and python outputs
- Cleanup excess parameters 

There is two way to generate the code:
    -Using the function 'cli_acetone' to directly generate both the output python and the code C
    -Using the class 'CodeGenerator' to have more controle on the generation

This method is mainly used as a command-line, either by runing the python file, either by using the built in command: acetone_generate.
Confere to the ReadMe for example using a terminal.
This method is prefered when using the package. 
It allows more regarding the type of the arguments, give more controle over the generation.